# Loan Approval Prediction Kaggle Competition
## October 28, 2024
DICHOSO, Aaron Gabrielle C.

This Notebook is part of a series of notebooks that will contain documentation and methods used for training a Logistic Regression Classifier (LRC) used in the <a href="https://www.kaggle.com/competitions/playground-series-s4e10/"><b>2024 Loan Approval Prediction Kaggle Playground Series</b></a>. 


For this notebook, I will focus on the methods that I utilized for model training and hyperparameter training.

To view the data cleaning itself, feel free to visit the following notebook: 

<ul>
    <li>1. Data Exploration, Cleaning, and Transformations</li>
</ul>

I chose to test the LRC because it inherently supports binary classification, which is exhibited by the dataset. Additionally, it is computationally efficient and low resource and will allow me to rapidly test it in my system. 

# 1. Import Cleaned Datasets

We first import the cleaned datasets from the previous notebook first. In this repository, the cleaned datasets are saved in the <b>./output</b> directory.

In [1]:
##Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

As observed, there are two kinds of training datasets used, the train set without oversampling, and the dataset that underwent ADASYN oversampling. I wish to test the performance of the model comparing these two methods during the hyperparameter tuning phase.

In [2]:
##Import Training Dataset
loans_train_df = pd.read_csv('./outputs/cleaned_loans_train.csv')
loans_train_df.head(5)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status,PERSON_HOME_OWNERSHIP_MORTGAGE,...,LOAN_GRADE_B,LOAN_GRADE_C,LOAN_GRADE_D,LOAN_GRADE_E,LOAN_GRADE_F,LOAN_GRADE_G,CB_PERSON_CRED_HIST_LENGTH_11_17,CB_PERSON_CRED_HIST_LENGTH_18_above,CB_PERSON_CRED_HIST_LENGTH_5_10,CB_PERSON_CRED_HIST_LENGTH_5_below
0,1.569797,-1.081318,-1.896898,-0.578305,0.390423,0.117380,0,1.719062,0,0,...,1,0,0,0,0,0,1,0,0,0
1,-0.921741,-0.052550,0.601227,-0.937769,0.896212,-0.973222,0,-1.364513,0,0,...,0,1,0,0,0,0,0,0,0,1
2,0.240977,-1.508084,0.923860,-0.578305,-0.470628,0.553620,0,1.185873,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0.407079,0.435878,1.579649,0.500086,0.277050,0.117380,0,0.087481,0,0,...,1,0,0,0,0,0,0,0,1,0
4,-0.921741,0.098465,-0.486519,-0.578305,-1.318902,-0.646041,0,-0.721995,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
loans_train_ada_df = pd.read_csv('./outputs/cleaned_loans_train_ada.csv')
loans_train_ada_df.head(5)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,PERSON_HOME_OWNERSHIP_MORTGAGE,PERSON_HOME_OWNERSHIP_OTHER,...,LOAN_GRADE_C,LOAN_GRADE_D,LOAN_GRADE_E,LOAN_GRADE_F,LOAN_GRADE_G,CB_PERSON_CRED_HIST_LENGTH_11_17,CB_PERSON_CRED_HIST_LENGTH_18_above,CB_PERSON_CRED_HIST_LENGTH_5_10,CB_PERSON_CRED_HIST_LENGTH_5_below,loan_status
0,1.569797,-1.081318,-1.896898,-0.578305,0.390423,0.117380,0,1.719062,0,0,...,0,0,0,0,0,1,0,0,0,0
1,-0.921741,-0.052550,0.601227,-0.937769,0.896212,-0.973222,0,-1.364513,0,0,...,1,0,0,0,0,0,0,0,1,0
2,0.240977,-1.508084,0.923860,-0.578305,-0.470628,0.553620,0,1.185873,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.407079,0.435878,1.579649,0.500086,0.277050,0.117380,0,0.087481,0,0,...,0,0,0,0,0,0,0,1,0,0
4,-0.921741,0.098465,-0.486519,-0.578305,-1.318902,-0.646041,0,-0.721995,0,0,...,0,0,0,0,0,0,0,0,1,0


# 2. Hyperparameter Tuning

The LRC has several hyperparameters that should be tuned to maximize its performance.

In this notebook, I will focus on tuning the following <a href="https://scikit-learn.org/dev/modules/generated/sklearn.linear_model.LogisticRegression.html">hyperparameters of the LRC</a>:
<ol>
    <li><b>C:</b> The inverse regularization strength</li>
    <li><b>tol:</b> The tolerance used for stopping the training</li>
    <li><b>solver:</b> The solver used for optimization</li>
    <li><b>oversampling_method:</b> The type of oversampling done in the dataset used.</li>
</ol>

The range of follows I chose for these hyperparameters are as follows:
<ol>
    <li><b>C:</b>[0.001, 100]</li>
    <li><b>tol:</b>[0.00001, 0.1]</li>
    <li><b>solver:</b> all the possible values for the solver specified in <a href="https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LogisticRegression.html">scikit-learn</a></li>
    <li><b>oversampling_method:</b> Either using the ADASYN oversampled data set or the unbalanced labels dataset.</li>

</ol>

In [4]:
df_hyper_tuning = pd.DataFrame(columns=['C', 'tol', 'solver', 'oversampling_method', 'roc_auc'])
df_hyper_tuning

,C,tol,solver,oversampling_method,roc_auc


For the specific method of hyperparameter tuning, I chose to perform bayesian optimization, which is a hyperparameter tuning method that involves observing the past iterations of the tuning process to influence the configurations to test later on. I chose this method over GridSearch because of the numerous possible configurations that I would need to search through not being a feasible method for my system. Additionally, I chose it over RandomSearch because bayesian optimization would be able to utilize my system resources more effectively by searching in areas with higher probabilities of giving me high performances as opposed to randomly testing configurations.

To perform bayesian optimization, I utilized the <a href="https://scikit-optimize.github.io/stable/modules/generated/skopt.gp_minimize.html">gp_minimize()</a> function provided by the scikit-optimize library.

Following the instructions found in the documentations, I first initialized the search space to be used in the optimization process. This involved creating an array that pertains to the hyperparameters to tune, the data type of the hyperparameters, and the range of possible values to test in the hyperparameter tuning process.

Afterwards, I created the objective function that the gp_minimize() function will execute. The objective function will use the search space defined earlier and test different values for the hyperparameters. It will then return the negative value of the Area Under the ROC (AUC) obtained from 3-fold cross validation, as this value will be minimized by the gp_minimize() function. Invalid configurations during hyperparameter tuning process will be given a positive value, allowing the bayesian optimization process to avoid such configurations.

I also limit the number of calls performed by the function due to my limited resources.

In [5]:
from sklearn.model_selection import cross_val_score
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
import numpy as np
import sys
# Define the search space
search_space = [
    Real(0.001, 100, name='C'),
    Real(0.00001, 0.1, name='tol'),
    Categorical(['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'], name='solver'),
    Categorical(['none', 'ada'], name='oversampling_method')
]

# Define your objective function (e.g., maximizing accuracy)
@use_named_args(search_space)
def objective_function(C, tol, solver, oversampling_method):
    print("================")
    print("Configuration:")
    print("C:", C)
    print("Tolerance:", tol)
    print("Solver:", solver)   
    print("Oversampling Method:", oversampling_method)
    print("----------------")
    try:
        if oversampling_method == 'none':
            X = loans_train_df.loc[:, loans_train_df.columns != "loan_status"]
            y = loans_train_df["loan_status"]
        elif oversampling_method == 'ada':
            X = loans_train_ada_df.loc[:, loans_train_ada_df.columns != "loan_status"]
            y = loans_train_ada_df["loan_status"]
            
        model = LogisticRegression(class_weight='balanced', C=C, tol=tol, solver=solver, max_iter=300, verbose=True)
        roc_auc = cross_val_score(model, X, y, cv=3, scoring='roc_auc').mean()

        print("Results:", -roc_auc)
        df_hyper_tuning.loc[len(df_hyper_tuning.index)] = [C, tol, solver, oversampling_method, roc_auc] 
        print("================")
        return -roc_auc
    except:
        print("Invalid Config")
        return 100000
        

# Perform Bayesian Optimization
res = gp_minimize(objective_function, search_space, n_calls=500)

# Print best parameters
print("Best parameters:", res.x)

Configuration:
C: 61.38813373267659
Tolerance: 0.07506231648759176
Solver: saga
Oversampling Method: none
----------------
convergence after 4 epochs took 0 seconds
convergence after 2 epochs took 0 seconds


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 3 epochs took 0 seconds
Results: -0.9039113112458258
Configuration:
C: 52.423256744926704
Tolerance: 0.07538787067366931
Solver: lbfgs
Oversampling Method: ada
----------------
Results: -0.7910640337424818
Configuration:
C: 14.57621637433194
Tolerance: 0.06082289193828174
Solver: sag
Oversampling Method: none
----------------


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 6 epochs took 0 seconds
convergence after 7 epochs took 0 seconds


...

Configuration:
C: 69.39780763325983
Tolerance: 0.06909560503723802
Solver: newton-cholesky
Oversampling Method: none
----------------
Newton iter=1
  Check Convergence
    1. max |gradient| 0.06405383968359588 <= 0.06909560503723802
    2. Newton decrement 0.23634586422551485 <= 0.06909560503723802
Newton iter=2
  Check Convergence
    1. max |gradient| 0.014705618009092036 <= 0.06909560503723802
    2. Newton decrement 0.02850677712352269 <= 0.06909560503723802
  Solver did converge at loss = 0.39173145913335333.
Newton iter=1
  Check Convergence
    1. max |gradient| 0.06467162758572388 <= 0.06909560503723802
    2. Newton decrement 0.23585981370554177 <= 0.06909560503723802
Newton iter=2
  Check Convergence
    1. max |gradient| 0.015049297191170789 <= 0.06909560503723802
    2. Newton decrement 0.029019715038483565 <= 0.06909560503723802
  Solver did converge at loss = 0.3913757818666197.
Newton iter=1
  Check Convergence
    1. max |gradient| 0.06241649380326511 <= 0.0690956050372

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Configuration:
C: 69.38193510882492
Tolerance: 0.06908673380429567
Solver: newton-cholesky
Oversampling Method: none
----------------
Newton iter=1
  Check Convergence
    1. max |gradient| 0.06405383958145905 <= 0.06908673380429567
    2. Newton decrement 0.2363458640096695 <= 0.06908673380429567
Newton iter=2
  Check Convergence
    1. max |gradient| 0.014705617919688808 <= 0.06908673380429567
    2. Newton decrement 0.02850677686983614 <= 0.06908673380429567
  Solver did converge at loss = 0.39173145980710744.
Newton iter=1
  Check Convergence
    1. max |gradient| 0.0646716274855292 <= 0.06908673380429567
    2. Newton decrement 0.23585981350392857 <= 0.06908673380429567
Newton iter=2
  Check Convergence
    1. max |gradient| 0.015049297104563904 <= 0.06908673380429567
    2. Newton decrement 0.02901971477674941 <= 0.06908673380429567
  Solver did converge at loss = 0.3913757825378156.
Newton iter=1
  Check Convergence
    1. max |gradient| 0.062416493700757185 <= 0.069086733804295

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Best parameters: [95.68017653731005, 0.058807524193284035, 'sag', 'none']


These tuning results are saved in a dataframe. The hyperparameter tuning results for all models can be viewed in the <b>./hyper_tuning</b> directory.

In [6]:
df_hyper_tuning.sort_values(by=['roc_auc'], ascending=False)

,C,tol,solver,oversampling_method,roc_auc
103,95.680177,0.058808,sag,none,0.904708
374,15.491306,0.053997,saga,none,0.904677
367,54.809968,0.021123,sag,none,0.904670
409,100.000000,0.015138,sag,none,0.904654
94,90.684544,0.007061,liblinear,none,0.904632
...,...,...,...,...,...
176,88.349953,0.071328,lbfgs,ada,0.791064
116,11.401321,0.076612,lbfgs,ada,0.791064
394,0.001000,0.100000,lbfgs,ada,0.791064
255,100.000000,0.100000,lbfgs,ada,0.791064


In [7]:
df_hyper_tuning.to_csv('hyper_tuning/lrc_hyper_tuning.csv', index=False, header=True, encoding='utf-8')

# 3. Results

The results showed that the following configuration produced the best results:

<ol>
   <li><b>C:</b> 95.680177</li>
    <li><b>tol:</b> 0.058808</li>
    <li><b>solver:</b> sag</li>
    <li><b>oversampling_method: </b>none</li>
</ol>

First, it can be seen that not performing oversampling generally performed better for the LRC as opposed to ADASYN. This may be due to how ADASYN generates synthetic data of the minority class that is harder to differentiate from the initial majority class. This, in turn, makes the classification problem harder for LRCs, considering that it generally performs well for linearly separable data. The introduction of oversampling may have led to the data being more non-linearly separable, and thus, causing a lower performance for LRCs.

In [18]:
print("ADASYN AUC:", df_hyper_tuning[df_hyper_tuning.oversampling_method == 'ada']['roc_auc'].mean())
print("None AUC:", df_hyper_tuning[df_hyper_tuning.oversampling_method == 'none']['roc_auc'].mean())

ADASYN AUC: 0.8686842180528066
None AUC: 0.901555980468623


Typically, the high C value may entail overfitting, but the validation score of the configuration says otherwise.

In the case of the LRC, having a high C value means lower regularization strengths, entailing a lower bias but a higher variance. 

Finally, according to the <a href="https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LogisticRegression.html">sci-kit learn documentations</a>, the "sag" solver is able to converge fast for datasets whose features are approximately on the same scale. The result of the hyperparameter tuning process to choose the "sag" solver may due to the data cleaning performed, as all of the continuous features in the data have been scaled via standardization. 

# 4. Exporting Model

The model with the best configuration found during the hyperparameter tuning process is saved in the <b>./outputs</b> directory.

In [8]:
clf = LogisticRegression(class_weight='balanced', C=res.x[0], tol=res.x[1], solver=res.x[2], verbose=True, max_iter=200)

In [9]:
if res.x[3] == 'none':
    X = loans_train_df.loc[:, loans_train_df.columns != "loan_status"]
    y = loans_train_df["loan_status"]
elif res.x[3] == 'ada':
    X = loans_train_ada_df.loc[:, loans_train_ada_df.columns != "loan_status"]
    y = loans_train_ada_df["loan_status"]
    
clf.fit(X,y)

# Calculate the ROC AUC score
roc_auc = cross_val_score(clf, X, y, cv=3, scoring='roc_auc').mean()
print("Validation AUC:", roc_auc)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 7 epochs took 0 seconds
convergence after 7 epochs took 0 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


convergence after 8 epochs took 0 seconds
convergence after 7 epochs took 1 seconds
Validation AUC: 0.9043585292763282


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Validation AUC from 3-fold cross validation: 0.9043585292763282


In [10]:
from joblib import dump
clf.fit(X,y)
dump(clf, './outputs/lrc_model.joblib')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 6 epochs took 0 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


['./outputs/lrc_model.joblib']

# 5. Fitting into Test Data

Finally, we can now generate the predictions made by the DTC on the test data. This is done by isolating the features of the test samples, forwarding it to the DTC for prediction, and appending the predicted class labels with the corresponding IDs of the test data. Predictions for the models can be found in the <b>./predictions</b> directory.

In [11]:
##Import Testing Dataset
loans_test_df = pd.read_csv('./outputs/cleaned_loans_test.csv')
loans_test_df

,id,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,PERSON_HOME_OWNERSHIP_MORTGAGE,...,LOAN_GRADE_B,LOAN_GRADE_C,LOAN_GRADE_D,LOAN_GRADE_E,LOAN_GRADE_F,LOAN_GRADE_G,CB_PERSON_CRED_HIST_LENGTH_11_17,CB_PERSON_CRED_HIST_LENGTH_18_above,CB_PERSON_CRED_HIST_LENGTH_5_10,CB_PERSON_CRED_HIST_LENGTH_5_below
0,58645,-0.755638,0.404383,-0.117198,2.836600,1.455666,2.189522,0,-1.364513,0,...,0,0,0,0,1,0,0,0,0,1
1,58646,-0.257331,1.127233,0.601227,0.140622,0.722635,-0.646041,1,-0.266122,1,...,0,1,0,0,0,0,0,0,0,1
2,58647,-0.257331,-1.418731,0.403331,-0.937769,1.748450,-0.318861,1,-1.364513,0,...,0,0,0,1,0,0,0,0,0,1
3,58648,0.905387,-0.300610,0.169270,-0.398573,-0.470628,-0.209801,0,0.620670,0,...,0,0,0,0,0,0,0,0,1,0
4,58649,-0.257331,1.259932,0.923860,1.039281,1.573370,-0.100741,1,-0.266122,1,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39093,97738,-0.921741,-1.332883,-0.486519,-1.117500,0.044689,-0.646041,0,-0.266122,1,...,1,0,0,0,0,0,0,0,0,1
39094,97739,-0.921741,-0.389963,0.601227,-0.398573,-1.782989,-0.100741,0,-0.721995,1,...,0,0,0,0,0,0,0,0,0,1
39095,97740,3.895232,0.098465,-1.896898,1.039281,-1.043084,0.989861,0,2.637868,1,...,0,0,0,0,0,0,0,1,0,0
39096,97741,-0.921741,-1.019656,0.169270,0.859550,1.425586,2.516703,1,-0.266122,1,...,0,0,1,0,0,0,0,0,0,1


In [12]:
X_test = loans_test_df.loc[:, loans_test_df.columns != "id"]
X_test

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,PERSON_HOME_OWNERSHIP_MORTGAGE,PERSON_HOME_OWNERSHIP_OTHER,...,LOAN_GRADE_B,LOAN_GRADE_C,LOAN_GRADE_D,LOAN_GRADE_E,LOAN_GRADE_F,LOAN_GRADE_G,CB_PERSON_CRED_HIST_LENGTH_11_17,CB_PERSON_CRED_HIST_LENGTH_18_above,CB_PERSON_CRED_HIST_LENGTH_5_10,CB_PERSON_CRED_HIST_LENGTH_5_below
0,-0.755638,0.404383,-0.117198,2.836600,1.455666,2.189522,0,-1.364513,0,0,...,0,0,0,0,1,0,0,0,0,1
1,-0.257331,1.127233,0.601227,0.140622,0.722635,-0.646041,1,-0.266122,1,0,...,0,1,0,0,0,0,0,0,0,1
2,-0.257331,-1.418731,0.403331,-0.937769,1.748450,-0.318861,1,-1.364513,0,0,...,0,0,0,1,0,0,0,0,0,1
3,0.905387,-0.300610,0.169270,-0.398573,-0.470628,-0.209801,0,0.620670,0,0,...,0,0,0,0,0,0,0,0,1,0
4,-0.257331,1.259932,0.923860,1.039281,1.573370,-0.100741,1,-0.266122,1,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39093,-0.921741,-1.332883,-0.486519,-1.117500,0.044689,-0.646041,0,-0.266122,1,0,...,1,0,0,0,0,0,0,0,0,1
39094,-0.921741,-0.389963,0.601227,-0.398573,-1.782989,-0.100741,0,-0.721995,1,0,...,0,0,0,0,0,0,0,0,0,1
39095,3.895232,0.098465,-1.896898,1.039281,-1.043084,0.989861,0,2.637868,1,0,...,0,0,0,0,0,0,0,1,0,0
39096,-0.921741,-1.019656,0.169270,0.859550,1.425586,2.516703,1,-0.266122,1,0,...,0,0,1,0,0,0,0,0,0,1


In [13]:
y_pred = clf.predict(X_test)

In [14]:
loans_predictions_df = loans_test_df["id"].copy(deep=True)
loans_predictions_df = loans_predictions_df.to_frame()
loans_predictions_df.insert(1, 'loan_status', y_pred, True)

In [15]:
loans_predictions_df

,id,loan_status
0,58645,1
1,58646,0
2,58647,1
3,58648,0
4,58649,1
...,...,...
39093,97738,0
39094,97739,0
39095,97740,0
39096,97741,1


In [16]:
loans_predictions_df.to_csv('predictions/lrc_predictions.csv', index=False, header=True, encoding='utf-8')